In [1]:
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 2 - 2.2 Cascata

### Geração de Massa

In [3]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.flagCartaoCreditoContratado':[0],
                             'solicitante.flagLimiteGarantidoContratado':[0],
                             'payloadHomol.intermediarias.clienteBlindado':[0],
                             'payloadHomol.intermediarias.clienteBloqueado':[0],
                            #  'payloadHomol.intermediarias.serasaInvalido':[0],
                             'payloadHomol.saidas.mensagemFiltro':['99. Accepted'],
                             'payloadHomol.intermediarias.semInfoScore':[0],
                             'solicitante.rendaLiquidaPicpayBatch':[10000]
                            })

In [4]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_Cascata_1 = sherlock.Gera_Massa(base_entrada)
massa_Cascata_1.gera_cenario('payloadHomol.saidas.mensagemFiltro',["-99999",
                                                           "02. Cadastro Irregular",
                                                           "03. Cadastro Negado ou Em Andamento",
                                                           "04. Conta Inativa ou Conta Teste PicPay",
                                                           "05. Sistema Operacional Defasado",
                                                           "06. CPF Invalido",
                                                           "07. Obito",
                                                           "09. Fraude PicPay",
                                                           "10. Chargeback PicPay",
                                                           "11. Idade <= 20 ou >= 80",
                                                           "12. Atraso PicPay",
                                                           "13. Atraso Original",
                                                           "14. Reneg ou FNV Ativo",
                                                           "15. Valor de Restritivo BVS >= 10% da Renda",
                                                           "16. Valor de Restritivo Serasa >= 10% da Renda",
                                                           "17. Quantidade de IFs = 0",
                                                           "18. Atraso Bacen >= 10% da Renda",
                                                           "19. Prejuizo Bacen >= 10% da Renda",
                                                           "20. Reneg Bacen",
                                                           "21. Saldo Financiado de Cartao > 70% do Limite",
                                                           "22. IU Cheque > 90%",
                                                           "23. AD Bacen",
                                                           "24. Desenrola",
                                                           "25. No hit Bureaus",
                                                           "99. Accepted"
                                                           ])
massa_Cascata_1 = massa_Cascata_1.get_massa_final()

massa_Cascata_2 = sherlock.Gera_Massa(base_entrada)
massa_Cascata_2.gera_cenario('solicitante.flagCartaoCreditoContratado',[0,1]) #2
massa_Cascata_2.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0,1]) #4
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.clienteBlindado',[0,1]) #8
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.clienteBloqueado',[0,1]) #16
# massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.serasaInvalido',[0,1]) #32
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.semInfoScore',[0,1]) #64
massa_Cascata_2.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[1000,-99999,0,600]) #256
massa_Cascata_2.gera_cenario('payloadHomol.saidas.mensagemFiltro',["-99999","24. Desenrola","25. No hit Bureaus","99. Accepted"]) # 1024
massa_Cascata_2 = massa_Cascata_2.get_massa_final()

#Empilha todos casos com 1 caso padrão.
massa_CascataFinal = sherlock.Gera_Massa(base_entrada)
massa_CascataFinal.empilha_massas([massa_Cascata_1,
                                   massa_Cascata_2
])
                        


massa_CascataFinal.set_bom_arqv()
massa_CascataFinal.converte_massa()

--------------------------------------
Quantidade de Linhas da Massa: 538
--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
listaResultado = rp.execucaoModular(massa_CascataFinal.get_massa_json(),rp.FiltroVisaoPolitica)

In [6]:
# Lógica para validar, pode-se trocar o nome da variável na linha 6 para exibir uma freq dos dominios que essa variável possui.

import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['saidas']['mensagemFiltroPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,99. Accepted,22
1,-99999,33
2,02. Cadastro Irregular,1
3,03. Cadastro Negado ou Em Andamento,1
4,04. Conta Inativa ou Conta Teste PicPay,1
5,05. Sistema Operacional Defasado,1
6,06. CPF Invalido,1
7,07. Obito,1
8,09. Fraude PicPay,1
9,10. Chargeback PicPay,1


In [7]:
# Lógica para validar, pode-se trocar o nome da variável na linha 6 para exibir uma freq dos dominios que essa variável possui.

import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['passouRegraFiltroPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha = freqzinha.sort_values(by=['valor'], ascending=True)
freqzinha

,valor,Freq
6,1,256
5,3,128
3,4,8
4,5,4
0,6,8
2,7,14
1,8,120


In [8]:
# Renomeação das colunas para o cabeçalho do RMA
massa_CascataFinal.rename_cabecalho()

# Adaptando alguns nomes de variaveis para a função de teste.
massa_CascataFinal.rename_variavel("payloadHomol.intermediarias.clienteBloqueado","clienteBloqueado")
massa_CascataFinal.rename_variavel("payloadHomol.intermediarias.clienteBlindado","clienteBlindado")
massa_CascataFinal.rename_variavel("flagCartaoCreditoContratado","payloadParam.solicitante.flagCartaoCreditoContratado")
massa_CascataFinal.rename_variavel("flagLimiteGarantidoContratado","payloadParam.solicitante.flagLimiteGarantidoContratado")
massa_CascataFinal.rename_variavel("rendaLiquidaPicpayBatch","payloadParam.solicitante.rendaLiquidaPicpayBatch")

In [9]:
massa_CascataFinal.atribui_expected(listaResultado,"payloadHomol.saidas.mensagemFiltroPolitica","saida.mensagemFiltroPolitica")
massa_CascataFinal.atribui_expected(listaResultado,"payloadHomol.saidas.statusDecisao","statusDecisao")

massa_CascataFinal.get_massa_dm()

,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.flagCartaoCreditoContratado,payloadParam.solicitante.flagLimiteGarantidoContratado,clienteBlindado,clienteBloqueado,payloadParam.mensagemFiltro,scoreInexistente,payloadParam.solicitante.rendaLiquidaPicpayBatch,result:saida.mensagemFiltroPolitica,expected:saida.mensagemFiltroPolitica,result:statusDecisao,expected:statusDecisao
0,1,,,0,0,0,0,99. Accepted,0,10000,,99. Accepted,,APROVADO
1,2,,,0,0,0,0,-99999,0,10000,,-99999,,NEGADO
2,3,,,0,0,0,0,02. Cadastro Irregular,0,10000,,02. Cadastro Irregular,,NEGADO
3,4,,,0,0,0,0,03. Cadastro Negado ou Em Andamento,0,10000,,03. Cadastro Negado ou Em Andamento,,NEGADO
4,5,,,0,0,0,0,04. Conta Inativa ou Conta Teste PicPay,0,10000,,04. Conta Inativa ou Conta Teste PicPay,,NEGADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,534,,,1,1,1,1,99. Accepted,1,0,,97. PPCard Credito Contratado,,NEGADO
534,535,,,1,1,1,1,-99999,1,600,,97. PPCard Credito Contratado,,NEGADO
535,536,,,1,1,1,1,24. Desenrola,1,600,,97. PPCard Credito Contratado,,NEGADO
536,537,,,1,1,1,1,25. No hit Bureaus,1,600,,97. PPCard Credito Contratado,,NEGADO


### Conversão da massa para CSV

In [10]:
massa_CascataFinal.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaFiltrosVisaoPoliticaV2.csv',index=False)

In [11]:
# Análise Divergencias
data_id = 94
print(json.dumps(listaResultado[data_id-1],indent=4))

{
    "marcacaoMesa": "",
    "etapa": "",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "idade": 30,
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "REGULAR",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 10000,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
    

#